In [1]:
import kagglehub
import pandas as pd
import numpy as np

/home/kaiserliche/ImSim/imsim/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### movielens 1M
path = kagglehub.dataset_download("odedgolden/movielens-1m-dataset")

In [3]:
rcolumns = ['userid', 'movieid', 'rating', 'timestamp']
ratings = pd.read_table(f"{path}/ratings.dat", names = rcolumns, sep = "::", encoding = "latin1", engine='python')

In [4]:
mcolumns = ['movieid','movie', 'genres']
movies = pd.read_table(f"{path}/movies.dat", names = mcolumns ,sep = "::", encoding = "latin1", engine='python')

In [5]:
ratings

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
movies

,movieid,movie,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
ratings = ratings[ratings['rating'] > 0]

In [8]:
mergedData = ratings.merge(movies, on="movieid")

In [9]:
mergedData

,userid,movieid,rating,timestamp,movie,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [10]:
unique_movie_ids = sorted(mergedData["movieid"].unique())
movie_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_movie_ids)}


In [11]:
mergedData['movieid'] = mergedData['movieid'].map(movie_id_mapping)

In [12]:
mergedData['userid'] = mergedData['userid'] - 1

In [13]:
mergedData.drop(["timestamp", "movie"], axis = 1,inplace= True)

In [14]:
mergedData

,userid,movieid,rating,genres
0,0,1104,5,Drama
1,0,639,3,Animation|Children's|Musical
2,0,853,3,Musical|Romance
3,0,3177,4,Drama
4,0,2162,5,Animation|Children's|Comedy
...,...,...,...,...
1000204,6039,1019,1,Comedy
1000205,6039,1022,5,Drama|Romance|War
1000206,6039,548,5,Comedy|Drama
1000207,6039,1024,4,Drama


In [15]:
genres_ = mergedData['genres'].values

In [16]:
genres = set()
for gen in genres_.tolist():
    gen_ = gen.split("|")
    for gen__ in gen_:
        genres.add(gen__)

In [17]:
sorted(list(genres))

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [18]:
genlookup = {}
for i, genre in enumerate(sorted(list(genres))):
    genlookup[genre] = i

In [19]:
genlookup

{'Action': 0,
 'Adventure': 1,
 'Animation': 2,
 "Children's": 3,
 'Comedy': 4,
 'Crime': 5,
 'Documentary': 6,
 'Drama': 7,
 'Fantasy': 8,
 'Film-Noir': 9,
 'Horror': 10,
 'Musical': 11,
 'Mystery': 12,
 'Romance': 13,
 'Sci-Fi': 14,
 'Thriller': 15,
 'War': 16,
 'Western': 17}

In [20]:
def convert(genrestr):
    gens = genrestr.split("|")
    arr = []
    for g in gens:
        arr.append(genlookup[g])
    return arr

In [21]:
mergedData["genres"] = mergedData["genres"].apply(convert)

In [28]:
mergedData

,userid,movieid,rating,genres
0,0,1104,5,[7]
1,0,639,3,"[2, 3, 11]"
2,0,853,3,"[11, 13]"
3,0,3177,4,[7]
4,0,2162,5,"[2, 3, 4]"
...,...,...,...,...
1000204,6039,1019,1,[4]
1000205,6039,1022,5,"[7, 13, 16]"
1000206,6039,548,5,"[4, 7]"
1000207,6039,1024,4,[7]


In [43]:
############ sample test and train data here ###############

### select 20% of the data from merged data
movie_counts = mergedData["movieid"].value_counts()
movies_with_multiple_ratings = movie_counts[movie_counts > 5].index  # Movies rated by multiple users

# Step 2: Filter dataframe to keep only those movies
df_filtered = mergedData[mergedData["movieid"].isin(movies_with_multiple_ratings)]

# Step 3: Randomly sample 20% of these ratings for test data
np.random.seed(42)  # For reproducibility
test_indices = df_filtered.sample(frac=0.2, random_state=42).index

# Step 4: Create train and test sets
test_df= mergedData.loc[test_indices].copy()
train_df = mergedData.drop(test_indices).copy()


In [44]:
train_user_movie_matrix = train_df.pivot(index='userid', columns='movieid', values='rating').fillna(0)
test_user_movie_matrix = test_df.pivot(index='userid', columns='movieid', values='rating').fillna(0)

In [45]:
train_user_movie_matrix

movieid,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
userid,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
test_user_movie_matrix

movieid,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
mergedData_exploded = train_df.explode("genres")

In [48]:
user_genre_matrix = mergedData_exploded.groupby(["userid", "genres"]).size().unstack(fill_value=0)

In [49]:
user_genre_matrix

genres,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
userid,,,,,,,,,,,,,,,,,,
0,4,5,17,19,12,0,0,13,3,0,0,13,0,5,3,2,2,0
1,49,16,0,0,21,7,0,64,0,1,2,0,2,22,13,28,13,3
2,20,20,2,2,24,0,0,4,1,0,3,0,1,4,4,5,1,5
3,17,4,0,1,0,1,0,6,1,0,3,0,0,1,7,4,2,2
4,24,8,3,4,41,17,4,82,0,1,8,2,5,26,12,30,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,124,64,26,46,204,42,8,293,16,15,55,25,25,99,133,116,29,12
6036,25,7,0,3,49,14,1,79,4,8,7,3,12,20,34,59,13,3
6037,2,1,3,1,11,0,0,7,0,0,2,0,0,5,1,0,3,0


In [50]:
train_user_movie_matrix

movieid,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
userid,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
### creating user interaction matrix based on genre
def get_interaction(userid, genres):
    return sum(train_user_movie_matrix.loc[userid, gen] for gen in genres if gen in user_genre_matrix.columns)


In [52]:
train_df["interaction"] = train_df.apply(lambda row: get_interaction(row["userid"], row["genres"]), axis=1)

In [53]:
user_interaction_matrix = train_df.pivot(index='userid', columns='movieid', values='interaction').fillna(0)


In [54]:
user_interaction_matrix

movieid,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
### save all

np.save("../data/user_interaction_matrix.npy", user_interaction_matrix.values)
np.save("../data/train_user_item_matrix.npy", train_user_movie_matrix.values)
np.save("../data/user_genre_matrix.npy", user_genre_matrix.values)
np.save("../data/test_user_item_matrix.npy", test_user_movie_matrix.values)

In [56]:
test_user_movie_matrix

movieid,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
test_df

,userid,movieid,rating,genres
572789,3508,3248,4,[4]
87891,576,1091,4,"[5, 7, 9]"
760428,4516,963,3,"[3, 4, 11]"
563716,3468,1090,5,"[7, 16]"
278121,1675,670,4,[4]
...,...,...,...,...
371776,2171,2514,1,"[10, 15]"
738720,4413,3550,4,"[0, 14]"
518245,3197,1979,5,"[10, 15]"
593888,3617,2884,4,[7]


In [59]:
test_df.to_csv("../data/test_df.csv")

In [60]:
test_df.shape

(199883, 4)

In [61]:
train_df.shape

(800326, 5)

In [62]:
mergedData.shape

(1000209, 4)